# E2LSH Pipeline評価 - DuckDB + E5モデル (v2: 本文データ)

## 目的

E2LSHをパイプラインに統合し、v2データセット（本文データ）でDuckDBのHNSW検索と比較評価する。

## v2データセットの特徴

- Wikipediaの本文テキストからエンベディング
- タイトルのみのv1より長いテキスト
- コサイン類似度がより狭い範囲に集中する傾向

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import time
import random

from src.db import VectorDatabase
from src.pipeline import HNSWSearcher, E2LSHCascadeSearcher, LSHCascadeSearcher
from src.lsh import SimHashGenerator

## 1. DuckDBからv2データ読み込み

In [2]:
# DuckDBからv2データ（本文データ）を読み込み
db_path = Path('../data/sample_vectors_v2.duckdb')
db = VectorDatabase(db_path=db_path)
db.initialize()

print(f'Total documents: {db.count():,}')

# 全データ取得
all_docs = db.get_all()
print(f'Columns: {all_docs.columns.tolist()}')

Total documents: 10,000


Columns: ['id', 'text', 'vector', 'simhash', 'lsh_chunks']


In [3]:
# サンプルデータ確認
df_sample = db.get_by_ids([0, 1, 2])
for i, row in df_sample.iterrows():
    print(f"ID {row['id']}: {row['text'][:60]}...")

ID 0: {{中華圏の事物
| 画像=[[File:People's Literature Publishing House (2...
ID 1: モレロス州<noinclude>
[[Category:Country aliasテンプレート|Morelos]]
</...
ID 2: {{Infobox Musician
| Name                = 碧井 れみ
| Img      ...


In [4]:
# ベクトルとメタデータを抽出
vectors = np.stack(all_docs['vector'].values).astype(np.float32)
texts = all_docs['text'].tolist()
ids = all_docs['id'].tolist()

print(f'Vectors shape: {vectors.shape}')
print(f'Norm range: [{np.linalg.norm(vectors, axis=1).min():.4f}, {np.linalg.norm(vectors, axis=1).max():.4f}]')

Vectors shape: (10000, 1024)
Norm range: [1.0000, 1.0000]


In [5]:
# コサイン類似度の分布確認
sample_indices = np.random.choice(len(vectors), min(1000, len(vectors)), replace=False)
sample_vecs = vectors[sample_indices]
similarities = sample_vecs @ sample_vecs.T
np.fill_diagonal(similarities, 0)
upper_tri = similarities[np.triu_indices_from(similarities, k=1)]

print(f'コサイン類似度分布（ランダムペア）:')
print(f'  Mean: {np.mean(upper_tri):.4f}')
print(f'  Std:  {np.std(upper_tri):.4f}')
print(f'  Range: [{np.min(upper_tri):.4f}, {np.max(upper_tri):.4f}]')

コサイン類似度分布（ランダムペア）:
  Mean: 0.7723
  Std:  0.0298
  Range: [0.6185, 0.9962]


## 2. 検索器の初期化

In [6]:
# HNSW検索器（DuckDB）
hnsw_searcher = HNSWSearcher(db)

# SimHash検索器（ベースライン比較用）
simhash_gen = SimHashGenerator(dim=1024, hash_bits=128, seed=42)
simhash_searcher = LSHCascadeSearcher(
    db=db,
    simhash_generator=simhash_gen,
    num_chunks=16,
    step2_top_n=500,
)

# E2LSH検索器
e2lsh_searcher = E2LSHCascadeSearcher(
    vectors=vectors,
    texts=texts,
    ids=ids,
    w=8.0,
    k=4,
    num_tables=8,
    seed=42,
)

print('検索器を初期化しました')
print(f'  SimHash: 128bit, 16chunks')
print(f'  E2LSH: w=8.0, k=4, L=8')

検索器を初期化しました
  SimHash: 128bit, 16chunks
  E2LSH: w=8.0, k=4, L=8


## 3. 単一クエリでの動作確認

In [7]:
# 最初のドキュメントをクエリとして使用
query_vec = vectors[0]
query_id = ids[0]
query_text = texts[0]

print(f'Query ID: {query_id}')
print(f'Query Text: {query_text[:80]}...')

Query ID: 0
Query Text: {{中華圏の事物
| 画像=[[File:People's Literature Publishing House (20230318142756).jpg|2...


In [8]:
# HNSW検索
hnsw_results, hnsw_time = hnsw_searcher.search(query_vec, top_k=10)

print('=== HNSW Results (Ground Truth) ===')
print(f'Time: {hnsw_time:.2f} ms')
for r in hnsw_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:40]}...')

=== HNSW Results (Ground Truth) ===
Time: 81.57 ms
  id=0, score=1.0000, text={{中華圏の事物
| 画像=[[File:People's Literature...
  id=9283, score=0.9133, text={{出典の明記|date=2018年4月17日 (火) 06:19 (UTC)}...
  id=3671, score=0.9106, text={{中華圏の事物
| 画像=[[ファイル:人民教育出版社 - panoramio...
  id=6035, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
| 簡体字=广西人民...
  id=9438, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
| 簡体字=江苏人民...


In [9]:
# SimHash検索
simhash_results, simhash_metrics = simhash_searcher.search(query_vec, top_k=10)

print('=== SimHash Cascade Results ===')
print(f'Total time: {simhash_metrics.total_time_ms:.2f} ms')
print(f'Step1 candidates: {simhash_metrics.step1_candidates}')
for r in simhash_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:40]}...')

=== SimHash Cascade Results ===
Total time: 386.06 ms
Step1 candidates: 9496
  id=0, score=1.0000, text={{中華圏の事物
| 画像=[[File:People's Literature...
  id=9283, score=0.9133, text={{出典の明記|date=2018年4月17日 (火) 06:19 (UTC)}...
  id=3671, score=0.9106, text={{中華圏の事物
| 画像=[[ファイル:人民教育出版社 - panoramio...
  id=9438, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
| 簡体字=江苏人民...
  id=5861, score=0.9080, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
| 簡体字=甘肃人民...


In [10]:
# E2LSH検索
e2lsh_results, e2lsh_metrics = e2lsh_searcher.search(query_vec, top_k=10)

print('=== E2LSH Cascade Results ===')
print(f'Total time: {e2lsh_metrics.total_time_ms:.2f} ms')
print(f'Step1 candidates: {e2lsh_metrics.step1_candidates}')
for r in e2lsh_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:40]}...')

=== E2LSH Cascade Results ===
Total time: 25.94 ms
Step1 candidates: 100
  id=0, score=1.0000, text={{中華圏の事物
| 画像=[[File:People's Literature...
  id=9283, score=0.9133, text={{出典の明記|date=2018年4月17日 (火) 06:19 (UTC)}...
  id=3671, score=0.9106, text={{中華圏の事物
| 画像=[[ファイル:人民教育出版社 - panoramio...
  id=6035, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
| 簡体字=广西人民...
  id=9438, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
| 簡体字=江苏人民...


In [11]:
# Recall比較
hnsw_ids = set(r.id for r in hnsw_results)
simhash_ids = set(r.id for r in simhash_results)
e2lsh_ids = set(r.id for r in e2lsh_results)

print('=== Recall@10 比較 ===')
print(f'SimHash: {len(hnsw_ids & simhash_ids) / len(hnsw_ids):.2f}')
print(f'E2LSH:   {len(hnsw_ids & e2lsh_ids) / len(hnsw_ids):.2f}')

=== Recall@10 比較 ===
SimHash: 0.60
E2LSH:   1.00


## 4. 複数クエリでの評価

In [12]:
# 100件のクエリをランダムサンプリング
random.seed(42)
num_queries = 100
query_indices = random.sample(range(len(vectors)), num_queries)

print(f'評価クエリ数: {num_queries}')

評価クエリ数: 100


In [13]:
def evaluate_all_searchers(query_indices, hnsw_searcher, simhash_searcher, e2lsh_searcher, vectors):
    """全検索器を評価"""
    results = {
        'hnsw_times': [],
        'simhash_times': [],
        'simhash_recalls': [],
        'simhash_candidates': [],
        'e2lsh_times': [],
        'e2lsh_recalls': [],
        'e2lsh_candidates': [],
    }
    
    for idx in query_indices:
        query_vec = vectors[idx]
        
        # HNSW (Ground Truth)
        hnsw_results, hnsw_time = hnsw_searcher.search(query_vec, top_k=10)
        hnsw_ids = set(r.id for r in hnsw_results)
        results['hnsw_times'].append(hnsw_time)
        
        # SimHash
        simhash_results, simhash_metrics = simhash_searcher.search(query_vec, top_k=10)
        simhash_ids = set(r.id for r in simhash_results)
        results['simhash_times'].append(simhash_metrics.total_time_ms)
        results['simhash_recalls'].append(len(hnsw_ids & simhash_ids) / len(hnsw_ids) if hnsw_ids else 0)
        results['simhash_candidates'].append(simhash_metrics.step1_candidates)
        
        # E2LSH
        e2lsh_results, e2lsh_metrics = e2lsh_searcher.search(query_vec, top_k=10)
        e2lsh_ids = set(r.id for r in e2lsh_results)
        results['e2lsh_times'].append(e2lsh_metrics.total_time_ms)
        results['e2lsh_recalls'].append(len(hnsw_ids & e2lsh_ids) / len(hnsw_ids) if hnsw_ids else 0)
        results['e2lsh_candidates'].append(e2lsh_metrics.step1_candidates)
    
    return results

# 評価実行
eval_results = evaluate_all_searchers(query_indices, hnsw_searcher, simhash_searcher, e2lsh_searcher, vectors)

In [14]:
# 結果サマリー
print('=' * 70)
print('     HNSW vs SimHash vs E2LSH 評価結果 (v2: 本文データ, 10,000件)')
print('=' * 70)
print(f"\n{'指標':<20} {'HNSW':>12} {'SimHash':>12} {'E2LSH':>12}")
print('-' * 58)
print(f"{'Recall@10':<20} {'1.000':>12} {np.mean(eval_results['simhash_recalls']):>12.3f} {np.mean(eval_results['e2lsh_recalls']):>12.3f}")
print(f"{'平均レイテンシ (ms)':<20} {np.mean(eval_results['hnsw_times']):>12.2f} {np.mean(eval_results['simhash_times']):>12.2f} {np.mean(eval_results['e2lsh_times']):>12.2f}")
print(f"{'平均候補数':<20} {len(vectors):>12} {np.mean(eval_results['simhash_candidates']):>12.0f} {np.mean(eval_results['e2lsh_candidates']):>12.0f}")
print('=' * 70)

     HNSW vs SimHash vs E2LSH 評価結果 (v2: 本文データ, 10,000件)

指標                           HNSW      SimHash        E2LSH
----------------------------------------------------------
Recall@10                   1.000        0.646        0.996
平均レイテンシ (ms)                18.17       327.56        27.45
平均候補数                       10000         8978          100


## 5. E2LSHパラメータチューニング

In [15]:
# 異なるパラメータでの評価
param_configs = [
    {'w': 4.0, 'k': 4, 'num_tables': 4},
    {'w': 4.0, 'k': 4, 'num_tables': 8},
    {'w': 4.0, 'k': 4, 'num_tables': 16},
    {'w': 8.0, 'k': 4, 'num_tables': 8},
    {'w': 8.0, 'k': 4, 'num_tables': 16},
    {'w': 16.0, 'k': 4, 'num_tables': 8},
]

param_results = []

print('E2LSH パラメータチューニング...')
for config in param_configs:
    searcher = E2LSHCascadeSearcher(
        vectors=vectors,
        texts=texts,
        ids=ids,
        **config,
        seed=42,
    )
    
    recalls = []
    times = []
    candidates = []
    
    for idx in query_indices:
        query_vec = vectors[idx]
        
        hnsw_results, _ = hnsw_searcher.search(query_vec, top_k=10)
        hnsw_ids = set(r.id for r in hnsw_results)
        
        e2lsh_results, metrics = searcher.search(query_vec, top_k=10)
        e2lsh_ids = set(r.id for r in e2lsh_results)
        
        recalls.append(len(hnsw_ids & e2lsh_ids) / len(hnsw_ids) if hnsw_ids else 0)
        times.append(metrics.total_time_ms)
        candidates.append(metrics.step1_candidates)
    
    param_results.append({
        'w': config['w'],
        'k': config['k'],
        'L': config['num_tables'],
        'recall': np.mean(recalls),
        'latency_ms': np.mean(times),
        'candidates': np.mean(candidates),
    })
    
    print(f"  w={config['w']}, k={config['k']}, L={config['num_tables']}: "
          f"Recall={np.mean(recalls):.3f}, Latency={np.mean(times):.2f}ms")

E2LSH パラメータチューニング...


  w=4.0, k=4, L=4: Recall=0.996, Latency=27.11ms


  w=4.0, k=4, L=8: Recall=0.996, Latency=27.29ms


  w=4.0, k=4, L=16: Recall=0.996, Latency=27.60ms


  w=8.0, k=4, L=8: Recall=0.996, Latency=27.42ms


  w=8.0, k=4, L=16: Recall=0.996, Latency=28.25ms


  w=16.0, k=4, L=8: Recall=0.996, Latency=27.41ms


In [16]:
# パラメータ比較表
df_params = pd.DataFrame(param_results)
df_params = df_params.sort_values('recall', ascending=False)
print('\n=== E2LSH パラメータ比較 ===')
print(df_params.to_string(index=False))


=== E2LSH パラメータ比較 ===
   w  k  L  recall  latency_ms  candidates
 4.0  4  4   0.996   27.110191       100.0
 4.0  4  8   0.996   27.289670       100.0
 4.0  4 16   0.996   27.597888       100.0
 8.0  4  8   0.996   27.424629       100.0
 8.0  4 16   0.996   28.252364       100.0
16.0  4  8   0.996   27.410892       100.0


## 6. SimHash vs E2LSH 詳細比較

In [17]:
# 各クエリのRecall分布を比較
print('=== Recall分布比較 ===')
print(f"\n{'パーセンタイル':<15} {'SimHash':>12} {'E2LSH':>12}")
print('-' * 41)
for p in [0, 25, 50, 75, 100]:
    simhash_p = np.percentile(eval_results['simhash_recalls'], p)
    e2lsh_p = np.percentile(eval_results['e2lsh_recalls'], p)
    print(f"P{p:<14} {simhash_p:>12.3f} {e2lsh_p:>12.3f}")

=== Recall分布比較 ===

パーセンタイル              SimHash        E2LSH
-----------------------------------------
P0                     0.100        0.800
P25                    0.400        1.000
P50                    0.700        1.000
P75                    0.900        1.000
P100                   1.000        1.000


In [18]:
# Recall=1.0のクエリ割合
simhash_perfect = sum(1 for r in eval_results['simhash_recalls'] if r == 1.0)
e2lsh_perfect = sum(1 for r in eval_results['e2lsh_recalls'] if r == 1.0)

print(f'\n完全一致 (Recall=1.0) のクエリ数:')
print(f'  SimHash: {simhash_perfect}/{num_queries} ({simhash_perfect/num_queries*100:.1f}%)')
print(f'  E2LSH:   {e2lsh_perfect}/{num_queries} ({e2lsh_perfect/num_queries*100:.1f}%)')


完全一致 (Recall=1.0) のクエリ数:
  SimHash: 24/100 (24.0%)
  E2LSH:   97/100 (97.0%)


## 7. 評価レポート

In [19]:
best_e2lsh = max(param_results, key=lambda x: x['recall'])

print('=' * 70)
print('          E2LSH Pipeline 評価レポート (v2: 本文データ)')
print('=' * 70)
print(f'''
【データセット】
  バージョン: v2 (Wikipedia本文データ)
  モデル: E5-large (multilingual-e5-large)
  件数: {len(vectors):,}
  次元: {vectors.shape[1]}

【v2データの特徴】
  - 本文テキストからエンベディング（v1はタイトルのみ）
  - コサイン類似度が狭い範囲に集中
  - SimHashの相関係数が低い（-0.3〜-0.4程度）

【性能比較】
  | 指標 | HNSW | SimHash | E2LSH (best) |
  |------|------|---------|-------------|
  | Recall@10 | 1.000 | {np.mean(eval_results['simhash_recalls']):.3f} | {best_e2lsh['recall']:.3f} |
  | 平均レイテンシ | {np.mean(eval_results['hnsw_times']):.2f} ms | {np.mean(eval_results['simhash_times']):.2f} ms | {best_e2lsh['latency_ms']:.2f} ms |
  | 候補数 | {len(vectors):,} | {np.mean(eval_results['simhash_candidates']):.0f} | {best_e2lsh['candidates']:.0f} |

【E2LSH最適パラメータ】
  w = {best_e2lsh['w']}
  k = {best_e2lsh['k']}
  L = {best_e2lsh['L']}

【結論】
  - E2LSHはSimHashより大幅に高いRecallを達成
  - v2データ（本文）でもE2LSHは高い精度を維持
  - 候補数を100件程度に抑えつつ高Recallを実現
''')
print('=' * 70)

          E2LSH Pipeline 評価レポート (v2: 本文データ)

【データセット】
  バージョン: v2 (Wikipedia本文データ)
  モデル: E5-large (multilingual-e5-large)
  件数: 10,000
  次元: 1024

【v2データの特徴】
  - 本文テキストからエンベディング（v1はタイトルのみ）
  - コサイン類似度が狭い範囲に集中
  - SimHashの相関係数が低い（-0.3〜-0.4程度）

【性能比較】
  | 指標 | HNSW | SimHash | E2LSH (best) |
  |------|------|---------|-------------|
  | Recall@10 | 1.000 | 0.646 | 0.996 |
  | 平均レイテンシ | 18.17 ms | 327.56 ms | 27.11 ms |
  | 候補数 | 10,000 | 8978 | 100 |

【E2LSH最適パラメータ】
  w = 4.0
  k = 4
  L = 4

【結論】
  - E2LSHはSimHashより大幅に高いRecallを達成
  - v2データ（本文）でもE2LSHは高い精度を維持
  - 候補数を100件程度に抑えつつ高Recallを実現



In [20]:
# データベース接続を閉じる
db.close()
print('Done!')

Done!
